# Tutorial: Rare Variant Association Testing with `cellink`

This tutorial demonstrates how to perform rare variant association testing (RVAT) in single-cell data using the `cellink` package. We focus on two major types of RVAT: burden tests and the Sequence Kernel Association Test (SKAT). Rare variants can be especially informative for understanding disease biology, but they require careful aggregation and testing approaches due to their low frequency. 

We’ll walk through how to:
- Prepare single-cell and genotype data using `DonorData` objects.
- Apply burden tests with multiple variant annotation-based weights.
- Combine p-values from multiple tests using ACAT.
- Perform SKAT for gene-level rare variant analysis.

This tutorial uses a subset of data from the 1k1k project, filtered to the **CD8 Naive** T cell type, and focuses on **chromosome 22** for runtime feasibility. It builds on earlier tutorials that cover pseudobulk expression and variant annotation, which should be reviewed for full context.

> Note: This notebook assumes that you have already run the variant annotation step as described in the [Variant Annotation Tutorial](./explore_annotations.ipynb).

## Environment Setup

We begin by importing necessary libraries and defining key parameters for our analysis. `cellink` provides utilities that extend `AnnData` to handle both donor-level genotype data and single-cell RNA-seq data efficiently.

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pandas as pd

In [11]:
import gc
from pathlib import Path
import warnings

import anndata as ad
import scanpy as sc
import dask.array as da
import numpy as np
from tqdm.auto import tqdm

import cellink as cl
from cellink._core import DAnn, GAnn
from cellink.tl._rvat import run_burden_test, run_skat_test, beta_weighting
from cellink.utils import column_normalize, gaussianize
from cellink.at.acat import acat_test
from cellink.datasets import get_onek1k

In [ ]:
DATA = Path(cl.__file__).parent.parent.parent / "docs/tutorials/data"

In [10]:
n_gpcs = 20
n_epcs = 15
batch_e_pcs_n_top_genes = 2000
chrom = 22
cis_window = 500_000
cell_type = "CD8 Naive"
celltype_key = "predicted.celltype.l2"
pb_gex_key = f"PB_{cell_type}"  # pseudobulk expression in dd.G.obsm[key_added]
original_donor_col = "donor_id"
min_percent_donors_expressed = 0.1
do_debug = False

## Load and Prepare Data

Here, we load a prepared dataset (`onek1k`) that includes genotype and expression information from human donors. We also extract gene annotations using Ensembl via `pybiomart`, which are essential for defining cis-windows during eQTL analysis.

In [12]:
dd = get_onek1k(config_path='../../src/cellink/datasets/config/onek1k.yaml')
dd

/Users/larnoldt/cellink_sample_data/onek1k/onek1k_cellxgene.h5ad already exists. Verifying checksum.
/Users/larnoldt/cellink_sample_data/onek1k/OneK1K.noGP.vcf.gz already exists. Verifying checksum.
/Users/larnoldt/cellink_sample_data/onek1k/OneK1K.noGP.vcf.gz.csi already exists. Verifying checksum.
/Users/larnoldt/cellink_sample_data/onek1k/gene_counts_Ensembl_105_phenotype_metadata.tsv.gz already exists. Verifying checksum.


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │      DonorData(981 x n_donors=10,595,884, n_cells_per_donor=[333-3,511], donor_id = 'donor_id')  │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 ×         │ AnnData object with n_obs × n_vars = 1,248,980 ×   │
│ 10,595,884                                         │ 36,469                                             │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     obs: 'orig.ident', 'nCount_RNA',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'pool_number', 'predicted.celltype.l2',            │
│ 'TYPED', 'TYPED_ONLY', 'contig', 'id', 'id_mask',  │ 'predicted.celltype.l2.score', 'age',              │
│ 'length', 'quality'                                │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     uns: 'kinship'                                 │     var: 'vst.mean', 'vst.variance',               │
│                                                    │ 'vst.variance.expected',                           │
│                                                    │ 'vst.variance.standardized', 'vst.variable',       │
│                                                    │ 'feature_is_filtered', 'feature_name',             │
│                                                    │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type'                   │
│     obsm: 'gPCs'                                   │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title'      │
│     varm: 'filter'                                 │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'                     │
│                                                    │     varm: 'PCs'                                    │
└────────────────────────────────────────────────────┴─────────────────────────────────────────

In [13]:
def _get_ensembl_gene_id_start_end_chr():
    from pybiomart import Server
    server = Server(host='http://www.ensembl.org')
    dataset = (server.marts['ENSEMBL_MART_ENSEMBL'].datasets['hsapiens_gene_ensembl'])
    ensembl_gene_id_start_end_chr = dataset.query(attributes=['ensembl_gene_id', 'start_position', 'end_position', 'chromosome_name'])
    ensembl_gene_id_start_end_chr = ensembl_gene_id_start_end_chr.set_index("Gene stable ID")
    ensembl_gene_id_start_end_chr = ensembl_gene_id_start_end_chr.rename(columns={
        "Gene start (bp)": GAnn.start,
        "Gene end (bp)": GAnn.end,
        "Chromosome/scaffold name": GAnn.chrom,
    })
    return ensembl_gene_id_start_end_chr

In [14]:
ensembl_gene_id_start_end_chr = _get_ensembl_gene_id_start_end_chr()
ensembl_gene_id_start_end_chr

,start,end,chrom
Gene stable ID,,,
ENSG00000210049,577,647,MT
ENSG00000211459,648,1601,MT
ENSG00000210077,1602,1670,MT
ENSG00000210082,1671,3229,MT
ENSG00000209082,3230,3304,MT
...,...,...,...
ENSG00000197989,28578536,28583132,1
ENSG00000229388,28643159,28648829,1
ENSG00000289291,28736044,28737670,1


In [15]:
dd.C.var = dd.C.var.join(ensembl_gene_id_start_end_chr)
dd.C.obs[DAnn.donor] = dd.C.obs[original_donor_col]
dd.G.obsm["gPCs"] = dd.G.obsm["gPCs"][dd.G.obsm["gPCs"].columns[:n_gpcs]]

## Normalize and Filter Expression Data

We normalize single-cell expression using total-count normalization and log transformation. We then filter the dataset to only include cells of a specific type (`CD8 Naive`). This step ensures we're analyzing homogeneous populations, which increases power in eQTL detection.

In [18]:
sc.pp.normalize_total(dd.C)
sc.pp.log1p(dd.C)
sc.pp.normalize_total(dd.C)

# are the expression pcs computed by pseudobulking across all cell types?
mdata = sc.get.aggregate(dd.C, by=DAnn.donor, func="mean")
mdata.X = mdata.layers.pop("mean")

sc.pp.highly_variable_genes(mdata, n_top_genes=batch_e_pcs_n_top_genes)
sc.tl.pca(mdata, n_comps=n_epcs)

dd.G.obsm["ePCs"] = mdata[dd.G.obs_names].obsm["X_pca"]

In [19]:
dd = dd[..., dd.C.obs[celltype_key] == cell_type, :].copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │        DonorData(980 x n_donors=10,595,884, n_cells_per_donor=[1-583], donor_id = 'donor_id')    │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 980 ×         │ AnnData object with n_obs × n_vars = 52,538 ×      │
│ 10,595,884                                         │ 36,469                                             │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     obs: 'orig.ident', 'nCount_RNA',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'pool_number', 'predicted.celltype.l2',            │
│ 'TYPED', 'TYPED_ONLY', 'contig', 'id', 'id_mask',  │ 'predicted.celltype.l2.score', 'age',              │
│ 'length', 'quality'                                │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     uns: 'kinship'                                 │     var: 'vst.mean', 'vst.variance',               │
│                                                    │ 'vst.variance.expected',                           │
│                                                    │ 'vst.variance.standardized', 'vst.variable',       │
│                                                    │ 'feature_is_filtered', 'feature_name',             │
│                                                    │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'start', 'end',  │
│                                                    │ 'chrom'                                            │
│     obsm: 'gPCs', 'ePCs'                           │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│     varm: 'filter'                                 │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'          

In [20]:
gc.collect()

23469

In [21]:
dd.aggregate(key_added=pb_gex_key, sync_var=True, verbose=True)
dd.aggregate(obs=["sex", "age"], func="first", add_to_obs=True)
dd

INFO:cellink._core.donordata:Aggregated X to PB_CD8 Naive
INFO:cellink._core.donordata:Observation found for 980 donors.


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │        DonorData(980 x n_donors=10,595,884, n_cells_per_donor=[1-583], donor_id = 'donor_id')    │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 980 ×         │ AnnData object with n_obs × n_vars = 52,538 ×      │
│ 10,595,884                                         │ 36,469                                             │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│                                                    │ 'pool_number', 'predicted.celltype.l2',            │
│                                                    │ 'predicted.celltype.l2.score', 'age',              │
│                                                    │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     var: 'vst.mean', 'vst.variance',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'vst.variance.expected',                           │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'vst.variance.standardized', 'vst.variable',       │
│ 'TYPED', 'TYPED_ONLY', 'contig', 'id', 'id_mask',  │ 'feature_is_filtered', 'feature_name',             │
│ 'length', 'quality'                                │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'start', 'end',  │
│                                                    │ 'chrom'                                            │
│     uns: 'kinship'                                 │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│     obsm: 'gPCs', 'ePCs', 'PB_CD8 Naive'           │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'          

In [22]:
print(f"{pb_gex_key} shape:", dd.G.obsm[pb_gex_key].shape)
print("dd.shape:", dd.shape)

keep_genes = ((dd.G.obsm[pb_gex_key] > 0).mean(axis=0) >= min_percent_donors_expressed).values
dd = dd[..., keep_genes]
print("after filtering")
print(f"{pb_gex_key} shape:", dd.G.obsm[pb_gex_key].shape)
print("dd.shape:", dd.shape)

PB_CD8 Naive shape: (980, 36469)
dd.shape: (980, 10595884, 52538, 36469)
after filtering
PB_CD8 Naive shape: (980, 12642)
dd.shape: (980, 10595884, 52538, 12642)


In [23]:
# alternative to dd[:, dd.G.var.chrom == str(chrom), :, dd.C.var.chrom == str(chrom)]
dd = dd.sel(G_var=dd.G.var.chrom == str(chrom), C_var=dd.C.var.chrom == str(chrom)).copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │         DonorData(980 x n_donors=136,776, n_cells_per_donor=[1-583], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 980 × 136,776 │ AnnData object with n_obs × n_vars = 52,538 × 317  │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│                                                    │ 'pool_number', 'predicted.celltype.l2',            │
│                                                    │ 'predicted.celltype.l2.score', 'age',              │
│                                                    │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     var: 'vst.mean', 'vst.variance',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'vst.variance.expected',                           │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'vst.variance.standardized', 'vst.variable',       │
│ 'TYPED', 'TYPED_ONLY', 'contig', 'id', 'id_mask',  │ 'feature_is_filtered', 'feature_name',             │
│ 'length', 'quality'                                │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'start', 'end',  │
│                                                    │ 'chrom'                                            │
│     uns: 'kinship'                                 │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│     obsm: 'gPCs', 'ePCs', 'PB_CD8 Naive'           │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'                     │
│     varm: 'filter'                                 │     varm: 'PCs'                         

## Step 1: Annotate Variants with Functional Information

To inform the weighting in burden tests, we annotate variants using the VEP tool. These annotations include predicted functional consequences (e.g., missense, stop gained), CADD scores, and distance to gene transcription start sites (TSS).

This information allows biologically meaningful prioritization of variants when aggregating their effects.

In [51]:
vep_annotation_file = DATA / "variant_annotation/variants_vep_annotated.txt"

In [52]:
cl.tl.add_vep_annos_to_gdata(vep_anno_file=vep_annotation_file, gdata=dd.G, dummy_consequence=True)
dd.G.uns["variant_annotation_vep"]

[2025-04-23 10:18:10,791] INFO:cellink.tl._annotate_snps_genotype_data: Preparing VEP annotations for addition to gdata
[2025-04-23 10:18:10,792] INFO:cellink.tl._annotate_snps_genotype_data: Reading annotation file /data/nasif12/home_if12/hoev/git/sc-genetics/docs/tutorials/data/variant_annotation/variants_vep_annotated.txt


[2025-04-23 10:18:11,152] INFO:cellink.tl._annotate_snps_genotype_data: Annotation file loaded
[2025-04-23 10:18:11,175] INFO:cellink.tl._annotate_snps_genotype_data: Annotation columns: ['snp_id', 'Location', 'Allele', 'gene_id', 'transcript_id', 'Feature_type', 'Consequence', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'IMPACT', 'DISTANCE', 'STRAND', 'FLAGS', 'BIOTYPE', 'CANONICAL', 'ENSP', 'SIFT', 'PolyPhen', 'gnomADe_AF', 'gnomADe_AFR_AF', 'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF', 'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'CADD_PHRED', 'CADD_RAW', 'TSSDistance']
[2025-04-23 10:18:13,899] INFO:cellink.tl._annotate_snps_genotype_data: Changing dtype of categorical columns ['snp_id', 'gene_id', 'transcript_id', 'PHENO', 'Codons', 'TSSDistance', 'gnomADe_EAS_AF', 'ENSP', 'gnomADe_ASJ_AF', 'IMPACT', 'Existing_variation', 'gnomADe_NFE_AF', 'DISTANCE', 'STRAND'

,,,Consequence_3_prime_UTR_variant,Consequence_5_prime_UTR_variant,Consequence_NMD_transcript_variant,Consequence_coding_sequence_variant,Consequence_downstream_gene_variant,Consequence_frameshift_variant,Consequence_inframe_deletion,Consequence_inframe_insertion,Consequence_intergenic_variant,Consequence_intron_variant,...,gnomADe_OTH_AF,SOMATIC,CADD_PHRED,cDNA_position,CLIN_SIG,BIOTYPE,FLAGS,gnomADe_AFR_AF,gnomADe_FIN_AF,PolyPhen
snp_id,gene_id,transcript_id,,,,,,,,,,,,,,,,,,,,,
22_16388891_G_A,ENSG00000230643,ENST00000447704,0,0,0,0,1,0,0,0,0,0,...,NaN,-,NaN,-,-,unprocessed_pseudogene,-,NaN,NaN,NaN
22_16388968_C_T,ENSG00000230643,ENST00000447704,0,0,0,0,1,0,0,0,0,0,...,NaN,-,NaN,-,-,unprocessed_pseudogene,-,NaN,NaN,NaN
22_16389525_A_G,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,0,0,...,NaN,-,NaN,78/118,-,unprocessed_pseudogene,-,NaN,NaN,NaN
22_16390411_G_A,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,0,0,...,NaN,-,NaN,-,-,unprocessed_pseudogene,-,NaN,NaN,NaN
22_16391555_G_C,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,0,0,...,NaN,-,NaN,-,-,unprocessed_pseudogene,-,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22_50796327_CCA_C,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,0,1,...,NaN,-,NaN,-,-,protein_coding,-,NaN,NaN,NaN
22_50798021_A_G,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,0,1,...,NaN,-,2.263,-,-,protein_coding,-,NaN,NaN,NaN
22_50798635_T_C,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,0,1,...,NaN,-,NaN,-,-,protein_coding,-,NaN,NaN,NaN


In [53]:
cl.tl.aggregate_annotations_for_varm(
    dd.G, "variant_annotation_vep", agg_type="first", return_data=True
)  # TODO change agg type

[2025-04-23 10:18:15,036] INFO:cellink.tl._annotate_snps_genotype_data: Aggregating using method: first


,gene_id,transcript_id,Consequence_3_prime_UTR_variant,Consequence_5_prime_UTR_variant,Consequence_NMD_transcript_variant,Consequence_coding_sequence_variant,Consequence_downstream_gene_variant,Consequence_frameshift_variant,Consequence_inframe_deletion,Consequence_inframe_insertion,...,gnomADe_OTH_AF,SOMATIC,CADD_PHRED,cDNA_position,CLIN_SIG,BIOTYPE,FLAGS,gnomADe_AFR_AF,gnomADe_FIN_AF,PolyPhen
snp_id,,,,,,,,,,,,,,,,,,,,,
22_16388891_G_A,ENSG00000230643,ENST00000447704,0,0,0,0,1,0,0,0,...,NaN,-,NaN,-,-,unprocessed_pseudogene,-,NaN,NaN,NaN
22_16388968_C_T,ENSG00000230643,ENST00000447704,0,0,0,0,1,0,0,0,...,NaN,-,NaN,-,-,unprocessed_pseudogene,-,NaN,NaN,NaN
22_16389525_A_G,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,...,NaN,-,NaN,78/118,-,unprocessed_pseudogene,-,NaN,NaN,NaN
22_16390411_G_A,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,...,NaN,-,NaN,-,-,unprocessed_pseudogene,-,NaN,NaN,NaN
22_16391555_G_C,ENSG00000230643,ENST00000447704,0,0,0,0,0,0,0,0,...,NaN,-,NaN,-,-,unprocessed_pseudogene,-,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22_50796327_CCA_C,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,...,NaN,-,NaN,-,-,protein_coding,-,NaN,NaN,NaN
22_50798021_A_G,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,...,NaN,-,2.263,-,-,protein_coding,-,NaN,NaN,NaN
22_50798635_T_C,ENSG00000100239,ENST00000395741,0,0,0,0,0,0,0,0,...,NaN,-,NaN,-,-,protein_coding,-,NaN,NaN,NaN


In [54]:
dd.G.varm["variant_annotation"].columns

Index(['gene_id', 'transcript_id', 'Consequence_3_prime_UTR_variant',
       'Consequence_5_prime_UTR_variant', 'Consequence_NMD_transcript_variant',
       'Consequence_coding_sequence_variant',
       'Consequence_downstream_gene_variant', 'Consequence_frameshift_variant',
       'Consequence_inframe_deletion', 'Consequence_inframe_insertion',
       'Consequence_intergenic_variant', 'Consequence_intron_variant',
       'Consequence_mature_miRNA_variant', 'Consequence_missense_variant',
       'Consequence_non_coding_transcript_exon_variant',
       'Consequence_non_coding_transcript_variant',
       'Consequence_protein_altering_variant',
       'Consequence_splice_acceptor_variant',
       'Consequence_splice_donor_5th_base_variant',
       'Consequence_splice_donor_region_variant',
       'Consequence_splice_donor_variant',
       'Consequence_splice_polypyrimidine_tract_variant',
       'Consequence_splice_region_variant', 'Consequence_start_lost',
       'Consequence_stop_gained

In [55]:
dd.G.uns["variant_annotation_vep"]["CADD_RAW"].describe()

count    32966.000000
mean         0.122248
std          0.542875
min         -1.705125
25%         -0.115726
50%          0.023902
75%          0.205452
max          9.936896
Name: CADD_RAW, dtype: float64

## Step 2: Burden Testing

We now perform burden tests, which evaluate whether the **aggregate effect** of rare variants within a gene is associated with a phenotype — in this case, gene expression in CD8 Naive cells. We use different weighting schemes allow us to prioritize variants differently based on functional scores or allele frequency, including:
- **CADD_RAW**: Raw CADD scores.
- **maf_beta**: Beta weighting based on minor allele frequency (MAF).
- **tss_distance**: Distance to the transcription start site (TSS).
- **tss_distance_exp**: Exponential weighting based on TSS distance.

In [56]:
burden_agg_fct = "sum"
run_lrt = True
annotation_cols = ["CADD_RAW", "maf_beta", "tss_distance", "tss_distance_exp"]

rare_maf_threshold = 0.05

### Filtering for Rare Variants
Burden tests focus on rare variants—typically those with minor allele frequency (MAF) below 0.01 or 0.05. We filter our genotype data accordingly to isolate variants in this frequency range.

In [58]:
dd = dd.sel(G_var=dd.G.var.maf < rare_maf_threshold).copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │          DonorData(981 x n_donors=39,454, n_cells_per_donor=[6-927], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 39,454  │ AnnData object with n_obs × n_vars = 259,012 × 413 │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│                                                    │ 'pool_number', 'predicted.celltype.l2',            │
│                                                    │ 'predicted.celltype.l2.score', 'age',              │
│                                                    │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     var: 'vst.mean', 'vst.variance',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'vst.variance.expected',                           │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'vst.variance.standardized', 'vst.variable',       │
│ 'TYPED', 'TYPED_ONLY', 'contig', 'id', 'id_mask',  │ 'feature_is_filtered', 'feature_name',             │
│ 'length', 'quality'                                │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'chrom',         │
│                                                    │ 'start', 'end', 'strand', 'gene_name'              │
│     uns: 'variant_annotation_vep'                  │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│     obsm: 'ePCs', 'PB_CD4 Naive', 'gPCs'           │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'                     │
│     varm: 'filter', 'variant_annotation'           │     varm: 'PCs'                         

### Custom MAF Weights

We add custom MAF weights commonly used in burden tests, such as `Beta(MAF, 1, 25)`. These weights prioritize rarer variants in the analysis. TSS distance weight as used in the SAIGE-QTL paper are added manually for each gene

In [59]:
dd.G.varm["variant_annotation"]["maf_beta"] = beta_weighting(dd.G.var["maf"])

### Run burden tests using each annotation individually for weighting

Next, we specify the covariate matrix `F`, which includes sex, age, genetic PCs, and expression PCs. This controls for potential confounders in the association analysis.
We then iterate over all genes on chromosome 22, define a cis-window of ±500kb, and run a burden test using each of the annotation-based weighting schemes.

In [60]:
# This specifies covariates/fixed effects
F = np.concatenate(
    [
        np.ones((dd.shape[0], 1)),
        dd.G.obs[["sex"]].values - 1,
        dd.G.obs[["age"]].values,
        dd.G.obsm["gPCs"].values,
        dd.G.obsm["ePCs"],
    ],
    axis=1,
)
F[:, 2:] = column_normalize(F[:, 2:])

In [61]:
results = []
if isinstance(dd.G.X, da.Array | ad._core.views.DaskArrayView):
    if dd.G.is_view:
        dd._G = dd._G.copy()
    dd.G.X = dd.G.X.compute()

if do_debug:
    warnings.filterwarnings("ignore", category=RuntimeWarning)

for gene, row in tqdm(dd.C.var.iterrows(), total=dd.shape[3]):
    Y = gaussianize(dd.G.obsm[pb_gex_key][[gene]].values + 1e-5 * np.random.randn(dd.shape[0], 1))

    start = max(0, row.start - cis_window)
    end = row.end + cis_window
    _G = dd.G[:, (dd.G.var.pos < end)]
    _G = _G[:, (_G.var.pos > start)]
    _G = _G[:, (_G.X.std(0) != 0)]
    _G = _G.copy()

    # TODO make strand aware
    _G.varm["variant_annotation"]["tss_distance"] = np.abs(row.start - _G.var["pos"])
    _G.varm["variant_annotation"]["tss_distance_exp"] = np.exp(-1e-5 * _G.varm["variant_annotation"]["tss_distance"])

    rdf = run_burden_test(
        _G, Y, F, gene, annotation_cols=annotation_cols, burden_agg_fct=burden_agg_fct, run_lrt=run_lrt
    )
    results.append(rdf)

rdf = pd.concat(results)
rdf

 62%|██████▏   | 255/413 [00:10<00:06, 24.62it/s]/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:57: RuntimeWarning: divide by zero encountered in divide
  n = 1.0 / (GG - np.einsum("ij,ij->j", FG, A0iFG))
/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:58: RuntimeWarning: invalid value encountered in multiply
  M = -n * A0iFG
/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:62: RuntimeWarning: invalid value encountered in multiply
  self.beta_g += n[:, None] * GY
 65%|██████▌   | 270/413 [00:11<00:05, 25.87it/s]/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:57: RuntimeWarning: divide by zero encountered in divide
  n = 1.0 / (GG - np.einsum("ij,ij->j", FG, A0iFG))
/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:58: RuntimeWarning: invalid value encountered in multiply
  M = -n * A0iFG
/data/nasif12/home_if12/hoev/git/sc-genetics/src/cellink/at/gwas.py:62: RuntimeWarning: invalid value e

,burden_gene,egene,weight_col,burden_agg_fct,pv,beta,betaste,lrt
0,ENSG00000128322,ENSG00000128322,CADD_RAW,sum,NaN,NaN,NaN,NaN
1,ENSG00000128322,ENSG00000128322,maf_beta,sum,0.749558,6.368263e-05,1.994932e-04,0.101903
2,ENSG00000128322,ENSG00000128322,tss_distance,sum,0.699738,4.189485e-08,1.086274e-07,0.148745
3,ENSG00000128322,ENSG00000128322,tss_distance_exp,sum,0.603931,1.962877e-03,3.783820e-03,0.269107
0,ENSG00000093072,ENSG00000093072,CADD_RAW,sum,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3,ENSG00000100292,ENSG00000100292,tss_distance_exp,sum,0.555883,1.736222e-03,2.947907e-03,0.346883
0,ENSG00000099968,ENSG00000099968,CADD_RAW,sum,NaN,NaN,NaN,NaN
1,ENSG00000099968,ENSG00000099968,maf_beta,sum,0.664269,-4.094791e-05,9.434422e-05,0.188379
2,ENSG00000099968,ENSG00000099968,tss_distance,sum,0.341478,-1.585172e-08,1.666411e-08,0.904875


## Step 3: Combine Multiple Tests with ACAT

To summarize evidence from multiple annotations, we use the **ACAT (Aggregated Cauchy Association Test)**. This meta-analysis method combines p-values from the burden tests per gene into a single statistic, improving power and interpretability.

In [62]:
combined = rdf.dropna(subset=["pv"]).groupby("egene")["pv"].agg(lambda x: acat_test(x.values)).reset_index()
combined.sort_values("pv")

,egene,pv
84,ENSG00000100219,2.287059e-14
290,ENSG00000212939,1.102047e-10
186,ENSG00000133460,1.428849e-04
198,ENSG00000167077,1.599335e-04
53,ENSG00000100075,4.714004e-04
...,...,...
22,ENSG00000099940,9.867149e-01
319,ENSG00000235954,9.891713e-01
244,ENSG00000185339,9.897443e-01
397,ENSG00000280007,9.976023e-01


## Step 4: SKAT

As an alternative to burden testing, we apply the Sequence Kernel Association Test (SKAT), which models the variance component of aggregated rare variants rather than assuming a unidirectional effect. SKAT is more robust when variant effects differ in direction or magnitude.

Currently, `cellink` supports SKAT with the standard Beta(MAF, 1, 25) weighting scheme.

In [63]:
results = []

for gene, row in tqdm(dd.C.var.iterrows(), total=dd.shape[3]):
    Y = gaussianize(dd.G.obsm[pb_gex_key][[gene]].values + 1e-5 * np.random.randn(dd.shape[0], 1))

    start = max(0, row.start - cis_window)
    end = row.end + cis_window
    _G = dd.G[:, (dd.G.var.pos < end)]
    _G = _G[:, (_G.var.pos > start)]
    _G = _G[:, (_G.X.std(0) != 0)]

    rdict = run_skat_test(_G, Y, F, gene)
    results.append(rdict)

rdf = pd.DataFrame(results)
rdf

  0%|          | 0/413 [00:00<?, ?it/s]

[2025-04-23 10:19:17,917] INFO:root: Starting optimization ...
[2025-04-23 10:19:17,958] INFO:root: Starting optimization ...
[2025-04-23 10:19:18,009] INFO:root: Starting optimization ...


  1%|          | 3/413 [00:00<00:17, 22.98it/s]

[2025-04-23 10:19:18,044] INFO:root: Starting optimization ...


[2025-04-23 10:19:18,100] INFO:root: Starting optimization ...
[2025-04-23 10:19:18,198] INFO:root: Starting optimization ...


...

[2025-04-23 10:19:40,824] INFO:root: Starting optimization ...
[2025-04-23 10:19:40,897] INFO:root: Starting optimization ...


 99%|█████████▉| 410/413 [00:23<00:00, 18.02it/s]

[2025-04-23 10:19:40,950] INFO:root: Starting optimization ...
[2025-04-23 10:19:41,002] INFO:root: Starting optimization ...
[2025-04-23 10:19:41,039] INFO:root: Starting optimization ...


100%|██████████| 413/413 [00:23<00:00, 17.80it/s]


,burden_gene,egene,weight_col,pv
0,ENSG00000128322,ENSG00000128322,maf_beta,0.342631
1,ENSG00000093072,ENSG00000093072,maf_beta,0.977279
2,ENSG00000261202,ENSG00000261202,maf_beta,0.967615
3,ENSG00000183785,ENSG00000183785,maf_beta,0.999691
4,ENSG00000185721,ENSG00000185721,maf_beta,0.332412
...,...,...,...,...
408,ENSG00000186716,ENSG00000186716,maf_beta,0.604184
409,ENSG00000198355,ENSG00000198355,maf_beta,0.971767
410,ENSG00000273096,ENSG00000273096,maf_beta,0.235938
411,ENSG00000100292,ENSG00000100292,maf_beta,0.445208


In [64]:
rdf

,burden_gene,egene,weight_col,pv
0,ENSG00000128322,ENSG00000128322,maf_beta,0.342631
1,ENSG00000093072,ENSG00000093072,maf_beta,0.977279
2,ENSG00000261202,ENSG00000261202,maf_beta,0.967615
3,ENSG00000183785,ENSG00000183785,maf_beta,0.999691
4,ENSG00000185721,ENSG00000185721,maf_beta,0.332412
...,...,...,...,...
408,ENSG00000186716,ENSG00000186716,maf_beta,0.604184
409,ENSG00000198355,ENSG00000198355,maf_beta,0.971767
410,ENSG00000273096,ENSG00000273096,maf_beta,0.235938
411,ENSG00000100292,ENSG00000100292,maf_beta,0.445208
